In [ ]:
import pandas as pd
import numpy as np
import datetime
import phonenumbers
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.style.use("ggplot")
from sqlalchemy import create_engine
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [ ]:
# Copy to working directory from ~/Library/Messages/chat.db
chat_db_location = "chat.db"

# Copy to working directory from ~/Library/Application Support/AddressBook/Sources
address_db_location = "AddressBook-v22.abcddb"

# Data Loading

In [ ]:
engine = create_engine(f"sqlite:///{chat_db_location}")
conn = engine.connect()
result = conn.execute(
    'SELECT T1.ROWID, text, handle_id, is_from_me, \
    datetime(date/1000000000 + strftime("%s", "2001-01-01") , "unixepoch", "localtime") as date, \
    T3.chat_identifier, T3.display_name, T4.id \
    FROM message T1 \
    INNER JOIN chat_message_join T2 \
        ON T1.ROWID=T2.message_id \
    INNER JOIN chat T3 \
        ON T3.ROWID=T2.chat_id \
    LEFT JOIN handle T4 \
        ON T1.handle_id=T4.ROWID \
    ORDER BY date'
)

df_msg = pd.DataFrame(result.fetchall(), columns=["id", "message", "handle_id", "is_from_me", "time", "chat_identifier", "chat_name", "sender"])

result = conn.execute(
    'SELECT T1.ROWID, T2.mime_type \
    FROM message T1 \
    INNER JOIN chat_message_join T3 \
        ON T1.ROWID=T3.message_id \
    INNER JOIN attachment T2 \
    INNER JOIN message_attachment_join T4 \
        ON T2.ROWID=T4.attachment_id \
        WHERE T4.message_id=T1.ROWID')
df_att = pd.DataFrame(result.fetchall(), columns=["id", "attachment"])
df_att

result = conn.execute(
    'SELECT T1.chat_identifier, T3.id \
    FROM chat T1 \
    INNER JOIN chat_handle_join T2 \
        ON T1.ROWID=T2.chat_id \
    INNER JOIN handle T3 \
        ON T2.handle_id=T3.ROWID'
)

chat_participants = pd.DataFrame(result.fetchall(), columns=["chat_identifier", "handle_id"]).groupby("chat_identifier")["handle_id"].apply(lambda x: x.unique().tolist()).to_dict()

df = df_msg.set_index('id').join(df_att.set_index('id'))
df['attachment'] = [1 if type(t) is str else 0 for t in df['attachment']]
df["time"] = pd.to_datetime(df["time"])
df["date"] = df["time"].dt.date
df.loc[df["is_from_me"] == 1, "sender"] = "me"
df = df.drop(["is_from_me", "handle_id"], axis=1)
df = df[~df["message"].isnull()]

sia = SIA()
df["polarity"] = df["message"].apply(lambda m: sia.polarity_scores(m)["compound"])
conn.close()

In [ ]:
engine = create_engine(f"sqlite:///{address_db_location}")
conn = engine.connect()
result = conn.execute(
    'SELECT ZFULLNUMBER, ZFIRSTNAME, ZLASTNAME \
    FROM ZABCDPHONENUMBER \
    LEFT JOIN ZABCDRECORD \
    ON ZABCDPHONENUMBER.ZOWNER = ZABCDRECORD.Z_PK'
)
out = result.fetchall()
conn.close()
addresses = {}
for address, first, last in out:
    if first is None and last is not None:
        name = last
    elif first is not None and last is None:
        name = first
    elif first is None and last is None:
        name = address
    else:
        name = f"{first} {last}"
    address = phonenumbers.format_number(phonenumbers.parse(address, 'US'), phonenumbers.PhoneNumberFormat.E164)
    addresses[address] = name

df["chat_name"] = np.where(df["chat_name"] == "",
                           [", ".join([addresses[x] if x in addresses else x for x in chat_participants[chatid]]) if chatid in chat_participants else chatid for chatid in df["chat_identifier"]],
                           df["chat_name"])

valid_number_mask = []
for sender in df["sender"]:
    try:
        phonenumbers.parse(sender)
        valid_number_mask.append(True)
    except phonenumbers.NumberParseException:
        valid_number_mask.append(False)
    
df["sender"] = np.where(valid_number_mask, [addresses[x] if x in addresses else x for x in df["sender"]], df["sender"])

In [ ]:
df["type"] = np.where(df["sender"].values == "me", "Sent", "Received")
df["hour"] = df["time"].dt.hour
df["weekday"] = df["time"].dt.weekday

---

# Basic Stats

## Counts

In [ ]:
df_sent_by_me = df[df["sender"] == "me"]
df_not_sent_by_me = df[df["sender"] != "me"]
print(f"{len(df)} messages")
print(f"{len(df.groupby('chat_identifier').count())} chats")

## Top 5 Most Sent (by Chat)

In [ ]:
df_sent_by_me["chat_name"].value_counts().head()

## Top 5 Most Received (by Chat)

In [ ]:
df_not_sent_by_me["chat_name"].value_counts().head()

## Distribution of Messages per Chat

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df.groupby(["chat_name", "type"])["message"].count().sort_values(ascending=False).hist(log=True, ax=ax)
ax.set_title("Message Distribution")
plt.show()

# Temporal Stats

---

## Hourly Messaging Frequency

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df.groupby(["hour", "type"]).count()["message"].unstack(level=0).T.plot.area(ax=ax, alpha=0.5)
ax.set_xticks(np.arange(24))
ax.set_xticklabels([f"{x} am" for x in np.roll(np.arange(1, 13), 1)] + [f"{x} pm" for x in np.roll(np.arange(1, 13), 1)])
ax.set_title("Hourly Messaging Frequency")
ax.set_xlabel("Hour")
ax.set_ylabel("Number of Messages")
ax.legend()
plt.show()

## Weekly Messaging Frequency

In [ ]:
df["weekday"] = df["time"].dt.weekday
fig, ax = plt.subplots(figsize=(20, 10))
df.groupby(["weekday", "type"]).count()["message"].unstack(level=0).T.plot.area(ax=ax, alpha=0.5)
ax.set_xticks(range(7))
ax.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
ax.set_title("Weekly Messaging Frequncy")
ax.set_xlabel("Hour")
ax.set_ylabel("Number of Messages")
ax.legend()
plt.show()

## Weekly Frequency Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
week = df.groupby(["hour", "weekday"]).count()["message"].unstack(level=0).values
im = ax.imshow(week, interpolation='nearest')
ax.set_xticks(range(24))
ax.set_yticks(range(7))
ax.set_xticklabels([f"{x} am" for x in np.roll(np.arange(1, 13), 1)] + [f"{x} pm" for x in np.roll(np.arange(1, 13), 1)])
ax.set_yticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
fig.colorbar(im, orientation="horizontal", label="Number of Messages")
ax.grid(False)
ax.set_title("Weekly Message Frequency Heatmap")
ax.set_xlabel("Hour")
ax.set_ylabel("Day")
plt.show()

# Sentiment Analysis

---

## Top 5 Message Polarity

In [ ]:
pd.set_option('display.max_colwidth', -1)
display(df.sort_values(by="polarity", ascending=False)[["message", "sender", "chat_name", "polarity"]].head())
pd.reset_option('display.max_colwidth')

## Bottom 5 Message Polarity

In [ ]:
pd.set_option('display.max_colwidth', -1)
display(df.sort_values(by="polarity", ascending=True)[["message", "sender", "chat_name", "polarity"]].head())
pd.reset_option('display.max_colwidth')

## Top 5 Chat Polarity

In [ ]:
pd.DataFrame(df.groupby(["chat_name"]).mean()["polarity"].sort_values(ascending=False).head())

## Bottom 5 Chat Polarity

In [ ]:
pd.DataFrame(df.groupby(["chat_name"]).mean()["polarity"].sort_values(ascending=True).head())

## Top 5 Sender Polarity

In [ ]:
pd.DataFrame(df.groupby(["sender"]).mean()["polarity"].sort_values(ascending=False).head())

## Bottom 5 Sender Polarity

In [ ]:
pd.DataFrame(df.groupby(["sender"]).mean()["polarity"].sort_values(ascending=True).head())

## Weekly Sentiment Heatmap

In [ ]:
df["weekday"] = df["time"].dt.weekday
fig, ax = plt.subplots(figsize=(20, 10))
week = df.groupby(["hour", "weekday"]).mean()["polarity"].unstack(level=0).values
im = ax.imshow(week, interpolation='nearest')
ax.set_xticks(range(24))
ax.set_yticks(range(7))
ax.set_xticklabels([f"{x} am" for x in np.roll(np.arange(1, 13), 1)] + [f"{x} pm" for x in np.roll(np.arange(1, 13), 1)])
ax.set_yticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
fig.colorbar(im, orientation="horizontal", label="Average Polarity")
ax.grid(False)
ax.set_title("Weekly Sentiment Heatmap")
ax.set_xlabel("Hour")
ax.set_ylabel("Day")
plt.show()